# PointNet

> Creating [PointNet](https://arxiv.org/abs/1612.00593). 

In [ ]:
#| default_exp models/pointnet

### NOTE: THIS PAGE NEEDS FORMATING AND ADDING GENERAL FUNCTIONALITY


In [ ]:
#| export
import torch
import torch.nn as nn
import torch.nn.init as init
import math

In [ ]:
#| export 

class LinearLayer(nn.Module):
    "A linear layer that can be either used as a shared-mlp or as a block in a fully connected network"
    def __init__(self, in_channels, out_channels, fc=False, use_norm=True, use_relu=True):
        super().__init__()

        # we can skip the bias if we use batch normalization
        bias = not use_norm

        layers = [nn.Linear(in_channels, out_channels, bias=bias) if fc else nn.Conv1d(in_channels, out_channels, kernel_size=1, bias=bias)]

        if use_norm:
            layers.append(nn.BatchNorm1d(out_channels))

        if use_relu:
            layers.append(nn.ReLU())

        self.net = nn.Sequential(*layers)

        # initializing the weights of the module
        self.init_weights()

    def init_weights(self):
        init.kaiming_uniform_(self.net[0].weight, a=math.sqrt(2))
        # bias initialization?

    def forward(self, x):
        return self.net(x)

In [ ]:
#| export 
#| hide

class MaxPool1D(nn.Module):
    "MaxPool1D"
    def forward(self, x):
        return x.max(dim=-1)[0]

In [ ]:
#| export 
#| hide

# regularization loss used in PointNet
def feature_transform_regularizer(trans):
    d = trans.size()[1]
    I = torch.eye(d)[None, :, :]
    if trans.is_cuda:
        I = I.cuda()
    loss = torch.mean(torch.norm(torch.bmm(trans, trans.transpose(2,1)) - I, dim=(1,2)))
    return loss

In [ ]:
#| export 

class TNET(nn.Module):
    "The TNET network that is part of the PointNet architecture"
    def __init__(self, in_features:int): 
        super().__init__()
        self.in_features = in_features
        self.tnet = nn.Sequential(
            # Initial feature aggregation
            LinearLayer(in_features, 64),
            LinearLayer(64, 128), 
            LinearLayer(128, 1024),
            # max pooling
            MaxPool1D(),
            # MLP to create the final matrix
            LinearLayer(1024, 512, fc=True),
            LinearLayer(512, 256, fc=True),
            LinearLayer(256, in_features * in_features, fc=True, use_norm=False, use_relu=False)
        )

        self.init_weights()
    
    def init_weights(self):
        with torch.no_grad():
            # we want the initial state of the network to be as close a possible to the identity matrix
            # so we need to set a high value at the bias of the element in the main diagonal and small values
            # to the rest of the biases and the weights of the output layer
            diag_indx = torch.arange(0, self.in_features * self.in_features-1, self.in_features+1)
            
            # setting the bias to be a small number
            self.tnet[-1].net[0].bias.fill_(0.00001)
            # setting the bias of the elements in the diagonal to be equal to 1.0
            self.tnet[-1].net[0].bias[diag_indx] = 1.0
            # aggigning a small value to the weights of the last layer
            self.tnet[-1].net[0].weight *= 0.0001 

    def forward(self, x):
        x = self.tnet(x)
        # reshape x to create a matrix
        x = x.reshape(-1, self.in_features, self.in_features)
        return x

In [ ]:
#| export 

class PointNetCore(nn.Module):
    'As PointNet core we consider the remains of the classification network if we remove the classification head. '
    def __init__(self, point_features:int): # values per point, usually the x,y,z coordinates 
        super().__init__()
        
        self.input_TNET = TNET(3)
        self.mlp1 = nn.Sequential(
            LinearLayer(3, 64),
            LinearLayer(64, 64)
        )
        self.feature_TNET = TNET(64)
        self.mlp2 = nn.Sequential(
            LinearLayer(64, 64), 
            LinearLayer(64, 128),
            LinearLayer(128, 1024)
        )

        # creating variables to store the m1, m2 matrices produced by the TNET
        self.m1 = None
        self.m2 = None

    def forward(self, x):
        
        # calculating the first orthogonal matrix
        # to transform input features
        m1 = self.input_TNET(x)
        x = m1 @ x # Multiply x with the transform matrix
        # passing through the first series of mlps
        x = self.mlp1(x)
        # Applying second feature transform
        m2 = self.feature_TNET(x)
        x = m2 @ x
        # passing through the second series of mlps
        x = self.mlp2(x)

        # storing the m1 and m2 matrices
        self.m1 = m1
        self.m2 = m2

        return x

    def get_TNET_matrices(self):
        return self.m1, self.m2

    def regularization_loss(self):
        return feature_transform_regularizer(self.m1) + feature_transform_regularizer(self.m2)

In [ ]:
#| export 

class PointNetCLSHead(nn.Sequential):
    'The classification head used in PointNet'
    def __init__(self, in_channels:int,  # feature dimension of the latent space vector 
                       num_classes:int): # number of classes
        super().__init__(
            LinearLayer(in_channels, 512, fc=True),
            LinearLayer(512, 256, fc=True),
            nn.Dropout(p=0.3),
            LinearLayer(256, num_classes, fc=True, use_norm=False, use_relu=False)
        )

In [ ]:
#| export 

class PointNetCLS(nn.Module):
    'PointNet variant for classification'
    def __init__(self, 
                 point_features:int, # values per point, usually the x,y,z coordinates 
                 num_classes:int):   # number of classes 
        super().__init__()
        
        # Feature extraction for each point
        self.core = PointNetCore(point_features)
        # Feature aggregation - Symmetric Function
        self.maxpool = MaxPool1D()
        # Classification head  
        self.cls_head = PointNetCLSHead(1024, num_classes)
    
    def regularization_loss(self):
        return self.core.regularization_loss()

    def forward(self, x):
        
        x = self.core(x)
        
        args = None
        if isinstance(x, tuple):
            args = x[1:]
            x = x[0]
        
        x = self.maxpool(x)
        x = self.cls_head(x)

        if args is not None:
            return x, *args

        return x

In [ ]:
#| export 

class PointNetSeg(nn.Module):
    "A variant of PointNet that outputs a feature vector for each point in the point cloud"
    def __init__(self, in_channels:int=3): # values per point, usually the x,y,z coordinates 
        super().__init__()
        
        self.input_TNET = TNET(in_channels)
        self.mlp1 = LinearLayer(in_channels, 64)
        self.mlp2 = LinearLayer(64, 128)
        self.mlp3 = LinearLayer(128, 128)

        self.feature_TNET = TNET(128)
        self.mlp4 = LinearLayer(128, 512)
        self.mlp5 = LinearLayer(512, 2048)

        self.maxpool = MaxPool1D()

        self.mlp6 = nn.Sequential(
            LinearLayer(3008, 256),
            LinearLayer(256, 256),
            LinearLayer(256, 128),
            LinearLayer(128, 3)
        )

        # creating variables to store the m1, m2 matrices produced by the TNET
        self.m1 = None
        self.m2 = None

    def forward(self, x):

        _, _, N = x.shape
        features = []
        
        m1 = self.input_TNET(x)
        x = m1 @ x # Multiply x with the transform matrix

        # passing through the first series of mlps
        x = self.mlp1(x)
        features.append(x)
        x = self.mlp2(x)
        features.append(x)
        x = self.mlp3(x)
        features.append(x)
        # Applying second feature transform
        m2 = self.feature_TNET(x)
        x = m2 @ x
        features.append(x)
        # passing through the second series of mlps
        x = self.mlp4(x)
        features.append(x)
        x = self.mlp5(x)
        x = self.maxpool(x)
        features.append(x.unsqueeze(-1).repeat(1,1,N))

        # concatenating the features
        feat = torch.cat(features, dim=1)

        out_points = self.mlp6(feat)

        # storing the m1 and m2 matrices
        self.m1 = m1
        self.m2 = m2

        return out_points.permute(0,2,1)

    def get_TNET_matrices(self):
        return self.m1, self.m2

    def regularization_loss(self):
        return feature_transform_regularizer(self.m1) + feature_transform_regularizer(self.m2)

In [ ]:
#| hide
model = PointNetSeg().cuda()
x = torch.rand(6, 3, 2048).cuda()
out = model(x)

print(out.shape)    

torch.Size([6, 2048, 3])
